# Simple Soccer Simulator

## About 

This notebook will demonstrate, through increasingly complex scenarios, why it's (generally) advantageous to make "safe" passes while playing soccer. This is something I <em>totally</em> didn't get as a kid, but it's been on my mind a lot lately. 

I thought about throwing a GUI behind this, but that's a lot of work, so I'm leaving it as-is for now. 

My ultimate goal? To convince you (and myself) that making safe passes, not losing possession, and carefully making your way up the field is important; that the difference between an 80% pass completion rate and an 85% pass completion rate is significant, especially when everyone on the team is on board and goes for the higher pass completion rate. 

## Simulating a pass

Ok, let's get this out of the way because everything else will build on it. Here's a small snippet of code that takes as input a person's pass completion rate (between 0 and 1) and uses a weighted random setup (#Ithinkthisworks) to determine if the pass was successful or not. Yeah, yeah, yeah, there are a lot of factors that determine if a pass is successful beyond the player's pass completion rate, but this is #SimpleSoccerSimulator, after all :) 

In [1]:
import random

# returns True if the pass is completed successfully, given R chance of successful pass completion
# returns False if the pass was unsuccessful
def make_pass(R): 
    if R >= 1 or R <=0:
        raise ValueError("Warning: R should be a float between 0 and 1 (exclusive).")
    temp = random.random()
    if temp > R: 
        return False
    return True

And below is a demo of how it works. We assume a 95% pass completion rate, make 1000 passes, and (hopefully) approximately 942 of them are successful. This doesn't actually show that this is happening truly randomly, so let's just hope I did it right. 

In [2]:
R = 0.95
a = []
for x in xrange(1000):
    a.append(make_pass(R))
    
print sum(a)  # number of True elements in the array (confirms that make_pass works as expected)

947


## Make N consecutive passes (simple)

Can we all agree that in order to score, we usually need to string together a few passes? Let's say we need to string together <code>N</code> passes before we can score (this is obviously an oversimplification - sometimes a goal happens after 1 pass or none or 30, but bear with me ...). Assuming everyone on the team has the same pass completion rate of <code>R</code> (again, this is oversimplified, I know), this will calculate the chance of successfully completing N passes (first function) and introduce randomness in a simulation using the make_pass function above (second function). 

## Am I wrong? 

Probably. And let's think about how/why in order to decide if the rest of my arguments are useful. 

Who knows if [this](http://thesportjournal.org/article/analysis-of-goal-scoring-patterns-in-the-2012-european-football-championship/) is a legit source? Not me, but it's got #data, and that gives me something to build on. 

According to that site, 
> Since the landmark work of Reep and Benjamin (42), many studies have focused on goal scoring patterns in various national and international football tournaments. Reep and Benjamin (42) showed that approximately 80% of goals scored were the result of a short sequence of three or less passes and that 1 in 10 shots tend to lead to a goal. More recently, Hughes and Franks (21) showed that in the 1990 and 1994 World Cup tournaments 84% and 80% of goals respectively came from possessions of four or less passes. In addition, 80% and 77% of the shots at goal were a result of a sequence of four or less passes.

Ok, so maybe you don't have to make <code>N</code> passes in a row in order to score, but I made this notebook before reading that, and you do (presumably) need to string together passes to make it down the field, so the following bit from that same site, which says that an overwhelming percentage of goals are scored from inside the penalty area, might help convince you that the rest of this exercise is interesting. 
> Other research has examined the position on the pitch from which goals are scored. In a recent study Wright et al. (50) showed that from 167 goals from English Premier League, 87% of goals were scored inside the penalty area which is similar to the 90% observed by Olsen (36) for the 1986 World Cup whereas Dufour (14) reported 80% for the 1990 World Cup. Yiannakos and Armatas (51) reported that 44.4% of goals scored were inside the penalty area, 35.2% inside the goal area, and 20.4% outside the penalty area, for the 2004 European Championship in Portugal. Finally, Hughes et al. (22) showed that successful teams in the 1986 Football World Cup made more attempts inside the penalty area in comparison to unsuccessful teams.

Another part of my argument that is missing is this: why not dribble? 

In [3]:
def chance_of_n_consecutive_passes_simple(N, R):
    return R**N

def simulate_n_consecutive_passes_simple(N, R):
    result_str = ""
    for a_pass in xrange(N):
        result_str += "Pass " + str(a_pass+1) + " of " + str(N)
        if make_pass(R):
            result_str += " completed.\n"
        else:
            result_str += " failed.\n"
            return (False, result_str)
    result_str += str(N) + " passes completed. Success!\n"
    return (True, result_str)

## How much difference does 5% make, anyway? 

Ok, so now we're getting into the #goodstuff. The following 3 lines give the chance of completing 5 passes if each player has a 95% vs 90% vs 80% pass completion rate. Notice that the 5% difference between 95% completion and 90% completion results in 77% vs 59% chance of completing 5 passes; that means that with a 90% pass completion rate, your team only connects 5 passes 77% of the times that they would have connected 5 passes if everyone had a 95% completion rate. And with each player having an 80% pass completion rate, which still still <em>sounds</em> good until you get into this math ... you're more likely to lose the ball while making 5 passes than to actually connect 5 passes in a row!

In [4]:
print chance_of_n_consecutive_passes_simple(5,.95)
print chance_of_n_consecutive_passes_simple(5,.9)
print chance_of_n_consecutive_passes_simple(5,.8)

0.7737809375
0.59049
0.32768


In [5]:
simulation = simulate_n_consecutive_passes_simple(3,.5)
print simulation[0]
print simulation[1]

False
Pass 1 of 3 completed.
Pass 2 of 3 failed.



In [6]:
def try_until_success(function, *args):
    success = False
    count = 0
    result_str = ""
    while not success: 
        count += 1
        result_str += "Try " + str(count) + ":\n"
        result = function(*args)
        success = result[0]
        result_str += result[1]
    return count

In [7]:
print try_until_success(simulate_n_consecutive_passes_simple, 10,.8)
print try_until_success(simulate_n_consecutive_passes_simple, 10,.9)

8
3


## Make N consecutive passes (complex)

Assuming that each person has chance <code>R_i</code> of completion.  

In [8]:
from operator import mul

def chance_of_n_consecutive_passes_complex(N, R):
    if len(R) != N:
        raise ValueError("Warning: R should be an array of length N!")
    return reduce(mul, R, 1)

def simulate_n_consecutive_passes_complex(N, R):
    if len(R) != N:
        raise ValueError("Warning: R should be an array of length N!")
    
    result_str = ""
    for a_pass in xrange(N):
        result_str += "Pass " + str(a_pass+1) + " of " + str(N)
        if make_pass(R[a_pass]):
            result_str += " completed.\n"
        else:
            result_str += " failed.\n"
            return (False, result_str)
    result_str += str(N) + " passes completed. Success!\n"
    return (True, result_str)


In [9]:
# confirms that the simple matches the complex in a simple case
print chance_of_n_consecutive_passes_complex(5, [0.5, 0.5, 0.5, 0.5, 0.5])
print chance_of_n_consecutive_passes_simple(5, 0.5)

0.03125
0.03125


In [10]:
print chance_of_n_consecutive_passes_complex(3,[.95, 0.95, 0.95])
print chance_of_n_consecutive_passes_complex(3,[.95, 0.8, 0.95])

0.857375
0.722


In [11]:
simulation = simulate_n_consecutive_passes_complex(3,[.95, 0.5, 0.95])
print simulation[0]
print simulation[1]

True
Pass 1 of 3 completed.
Pass 2 of 3 completed.
Pass 3 of 3 completed.
3 passes completed. Success!



In [12]:
print try_until_success(simulate_n_consecutive_passes_complex, 10,[.9, .9, .9, .9, .9, .9, .9, .9, 0.8, 0.9])
print try_until_success(simulate_n_consecutive_passes_complex, 10,[.9, .9, .9, .9, .9, .9, .9, .9, 0.9, 0.9])

2
5


## Create teams, where each player has a pass completion rate

In [13]:
DELTA = 0.1  # equivalent to needing 10 passes to score

class Player:
    def __init__(self, name, (completion_rate, total_passes)):
        if completion_rate < 0:
            completion_rate = 0.01
        self.completion_rate = min(completion_rate, 0.98)
        self.total_passes = total_passes
        self.name = name
        
    def get_completion_rate(self):
        return self.completion_rate
        
    def update(self, pass_bool):
        return ## comment this out if you want the pass completion rate to dynamically change as the players play
        successful_passes = self.completion_rate * self.total_passes 
        if pass_bool:
            successful_passes += 1
        self.total_passes += 1
        self.completion_rate = successful_passes*1.0 / self.total_passes
        if self.completion_rate > 0.98:
            self.completion_rate = 0.98
                        
class Team:
    def __init__(self, name, max_players=11):
        self.players = []
        self.score = 0
        self.max_players = max_players
        self.name = name
        
    def __init__(self, name, players, max_players=11):
        self.players = players
        if len(players) > 11:
            raise ValueError("Too many players on the field!")
        self.score = 0
        self.max_players = max_players
        self.name = name
    
    def add_player(self, stats):
        if len(self.players) == self.max_players:
            raise ValueError("Too many players on the field!")
        self.players.append(Player(stats))
    
    def update_score(self):
        self.score += 1
        
class Game:
    def __init__(self, teamA, teamB):
        self.teamA = teamA
        self.teamB = teamB
        self.ball_position = 0
        self.possession = random.choice(self.teamA.players)
        self.possessing_team = self.teamA
        
        self.consecutive_passes = 0
    
    def get_winner(self):
        if self.teamA.score > self.teamB.score:
            return self.teamA
        elif self.teamB.score > self.teamA.score:
            return self.teamB
        else:
            return None
        
    def print_status(self):
        
        print "-1 ||", self.teamA.name, "-->         <--", self.teamB.name, "|| 1"
        
        print self.teamA.name, "score:", self.teamA.score
        print self.teamB.name, "score:", self.teamB.score
        
        print 'Ball at:', self.ball_position
        
        print 'Possession:', self.possession.name
        print 'Possessing team:', self.possessing_team.name
        print 'Consecutive passes:', self.consecutive_passes
        
    def update_ball_position(self, delta):
        self.ball_position += delta
        

    def step(self):
        
        pass_success = make_pass(self.possession.completion_rate)
        #print 'successful pass?', pass_success
        self.possession.update(pass_success)

        if self.teamA == self.possessing_team:
            
            self.update_ball_position(DELTA)

            if pass_success: 
                self.consecutive_passes += 1
                #print 'pass succeeded'
                other_players = list(self.teamA.players)
                other_players.remove(self.possession)
                #for p in other_players:
                #    print p.name
                self.possession = random.choice(other_players)
                #print self.possession.name
            else: 
                self.consecutive_passes = 0 
                #print 'pass failed'
                self.possession = random.choice(self.teamB.players)
                self.possessing_team = self.teamB
                #print self.possessing_team.name
                #print self.possession.name

                
        else: 
            self.update_ball_position(0-DELTA)

            if pass_success:
                self.consecutive_passes += 1 
                other_players = list(self.teamB.players)
                other_players.remove(self.possession)
                self.possession = random.choice(other_players)
            else:
                self.consecutive_passes = 0 
                
                self.possession = random.choice(self.teamA.players)
                self.possessing_team = self.teamA
                
        
        if self.ball_position > 1:
            self.teamA.update_score()
            self.ball_position = 0
            self.possession = random.choice(self.teamB.players)
            self.possessing_team = self.teamB
            #print 'Required', self.consecutive_passes, 'consecutive passes to score'
            self.consecutive_passes = 0 

        elif self.ball_position < -1:
            self.teamB.update_score()
            self.ball_position = 0
            self.possession = random.choice(self.teamA.players)
            self.possessing_team = self.teamA
            #print 'Required', self.consecutive_passes, 'consecutive passes to score'
            self.consecutive_passes = 0



In [14]:
p1 = Player('p1', (0.9,10))
p2 = Player('p2', (0.9,10))
p3 = Player('p3', (0.85,10))

p4 = Player('p4', (0.9,10))
p5 = Player('p5', (0.9,10))
p6 = Player('p6', (0.9,10))


t1 = Team('t1', [p1,p2,p3], 3)
t2 = Team('t2', [p4,p5,p6], 3)

game = Game(t1, t2)

In [15]:
game.print_status()

-1 || t1 -->         <-- t2 || 1
t1 score: 0
t2 score: 0
Ball at: 0
Possession: p3
Possessing team: t1
Consecutive passes: 0


In [16]:
#game.step()
#game.print_status()

In [17]:
for x in xrange(100):
    game.step()
game.print_status()
game.get_winner().name

-1 || t1 -->         <-- t2 || 1
t1 score: 3
t2 score: 2
Ball at: 0.5
Possession: p2
Possessing team: t1
Consecutive passes: 11


't1'

In [18]:
from collections import Counter

winners = []
for g in xrange(1000):
    game = Game(Team('t1', [p1,p2,p3], 3), Team('t2', [p4,p5,p6], 3))
    for time in xrange(100):
        game.step()
    winner = game.get_winner()
    if winner == None:
        winners.append('none')
    else: 
        winners.append(winner.name)
        
Counter(winners)

Counter({'none': 287, 't1': 319, 't2': 394})

Ok, but what about playing <b>for the fun of the game!</b> Most of us are playing adult co-ed soccer, after all. 

Do you have more fun when your team has the ball or when the other team has the ball? Next, I'll do some calculations to show that we're more likely to have more possession over the course of the game (e.g., each individual player is more likely to touch it more times) if we all have a better pass completion rate than if <em>even one</em> player decides to put all their eggs in the immediate fun gratification basket and do something risky every time they get the ball. 

http://tempofreesoccer.blogspot.com/2013/06/passing-passes-per-turnover-and.html

TODO: 

1. set "safe" "normal" "risky" pass completion rates for N players and the % chance that a given player will make the choice to carry out each of these 3 pass types
2. simulation differences are not in the pass completion rates but in the % chances (e.g., one team full of players that consistently make safe/normal passes vs. a team that consistently makes risky passes) 
3. pre-define that safe passes go back a little, risky passes go forward more than normal

Can I "show" that teams that make more safe/normal passes win more often than teams that make more risky passes? 

Something about shooting likelyhood? Incorporating a "choice" of making a shot from distance X from goal (where the farther you get the harder the shot is to make). Simulating punting (e.g., if keeper gets it, they will punt it and the ball re-starts at the midfield with 50/50 chance of going to your team ... or other options off the punt maybe?). 